<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 使用 Pydantic 生成结构化输出

In [ ]:
from videos.walkthroughs import walkthrough_42 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将通过结合 Pydantic 类和 LangChain 的 `JsonOutputParser` 大幅提升生成结构化输出的能力。

---

## 目标

完成这个 notebook 后，您将：

- 理解当前生成结构化数据方法的局限性。
- 学会使用 Pydantic 创建面向类的结构化数据生成。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.pydantic_v1 import BaseModel, Field

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 当前结构化数据方法的局限性

您的实现可能略有不同，不过我们在之前 notebook 练习的参考答案中采用了以下提示模板成功生成了一个包含书籍详情的 JSON 对象。

In [ ]:
book_template = ChatPromptTemplate.from_template('''\
Make a JSON object representing the details of the following book: {book_title}. \
It should have fields for:
- The title of the book.
- The author of the book.
- The year the book was originally published.

Only return the JSON. Never return non-JSON text including backtack wrappers around the JSON.''')

使用这个模板，我们的参考答案实现生成了以下书籍详情列表：

```python
[{'title': 'Dune', 'author': 'Frank Herbert', 'year_of_publication': 1965},
 {'title': 'Neuromancer', 'author': 'William Gibson', 'year': 1984},
 {'title': 'Snow Crash', 'author': 'Neal Stephenson', 'yearPublished': '1992'},
 {'title': 'The Left Hand of Darkness',
  'author': None,
  'publication_year': None},
 {'title': 'Foundation', 'author': 'Isaac Asimov', 'year': '1951'}]
 ```

得到的格式很好，但仔细一看，我们可以发现一些问题：

- 不是所有值的键名都一致，比如 `'year_of_publication'`、`'year'` 和 `'yearPublished'`。
- 年份有时以字符串形式生成（`'1992'`），有时为整数（`1984`），有时为 NoneType。

在课程的这个阶段，您可能已经对如何解决这些问题充满了想法。也许会想到这些：

- 在提示词中更具体地说明键的名称、值的类型，以及当 LLM 无法为某个字段生成数据时该怎么处理。
- 尝试添加系统消息，以更强烈地强调我们希望 LLM 如何生成响应。
- 提供几个示例，帮助模型理解它应该做什么和不应该做什么。

如果您在考虑这些，那真是太棒了，以这种方式解决问题的思路是对的。

但让我们考虑一下，更复杂的任务：

- 如果我们想要将更多的提示词模板化，比如应该包含哪些字段，应该怎么办？
- 如果数据结构变得更加复杂呢？
- 如果我们想捕捉数据类型的定义以便在其它地方使用呢？

再次强调，考虑到您已经知道的内容，您可能已经能想到可行的方法来完成每一个任务，尽管或许会变得相当复杂。幸运的是，LangChain 提供了多种工具来帮助我们生成结构化数据，使用它们会大大简化应用代码，并让我们更轻松地执行复杂的结构化数据生成任务。

---

## 结构化数据作为一个类

在我们借助 LangChain 特定工具来帮助生成结构化数据之前，先退一步，思考一下如何不在 LLM 的上下文中用 Python 描述一个数据结构。一种非常合理的方法是创建 Python 类。

这里我们定义一个 `Book` 类，以捕捉我们希望在提示模板中描述的内容。

In [ ]:
class Book:
    """Information about a book."""
    
    def __init__(self, title, author, year_of_publication):
        self.title = title
        self.author = author
        self.year_of_publication = year_of_publication

不过，我们刚才讨论的关于结构化数据的一些细节，这个类还没有体现出来，比如每个字段的类型。另外，这里没有关于每个字段应该包含什么的描述。

稍微改进一下，把这个类重写以加入 Python 类型提示，再加上一些评论来说明每个字段的预期值。

In [ ]:
class Book:
    """Information about a book."""

    def __init__(self, title: str, author: str, year_of_publication: int):
        self.title: str = title  # The title of the book
        self.author: str = author  # The author of the book
        self.year_of_publication: int = year_of_publication  # The year the book was published

它还缺少一些信息，比如默认值和数据验证，不过大体来说，如果我们有办法把上面类中包含的信息（包括注释）传递到提示词中，那可能就得心应手了。

---

## Pydantic

实际上，LangChain 提供了将类的信息传递给提示词的一切工具。这样，我们就可以清晰地表达希望生成的数据结构，然后让 LangChain 帮忙处理一些繁琐的工作，将在类中蕴含的信息传达给提示词。

不过，为了实现这一点，我们需要使用 Pydantic 类，而不是普通的 Python 类。

如果您没用过，[Pydantic](https://docs.pydantic.dev/latest/) 是“Python 中使用最广泛的数据验证库”。如果您在面向对象的 Python 代码中没有使用 Pydantic，很可能会想学习如何使用它。

我们只会使用 Pydantic 来构建简单的类，以便 LangChain 能利用类定义来创建有助于生成结构化数据的提示词。

相关的 Pydantic 功能已集成到 LangChain 中，使用 Pydantic 类之前需要先导入以下库。

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

导入 `BaseModel` 和 `Field` 后，就可以用 Pydantic 重写 `Book` 类，如下所示。

In [ ]:
class Book(BaseModel):
    """Information about a book."""

    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    year_of_publication: str = Field(description="The year the book was published")

正如您所看到的，构建 Pydantic 类要创建一个继承自 `BaseModel` 的类，像上面所做的那样。

我们不需要创建一个 `__init__` 函数，而是可以在类定义的顶层通过定义 `Field` 来提供类的字段，这样一来，就可以方便地提供一个 `description` 参数来说明字段的预期用法。

---

## 从类到格式化指令

为了将 Pydantic `Book` 类中定义的结构转化为 JSON 对象，我们需要一个提示词来为模型提供指导。LangChain 的 `JsonOutputParser` 就能满足这个需求。

首先导入 `JsonOutputParser` 类。

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

和之前用的 `StrOutputParser` 及 `SimpleJsonOutputParser` 解析器一样，我们需要创建一个解析器实例来在我们的工作流中使用。

不过，与之前的解析器不同的是，我们可以给 `JsonOutputParser` 提供一个参数 `pydantic_object`，并传入一个 Pydantic 对象，来表达我们希望 JSON 如何被解析。这里传入 Pydantic `Book`。

In [ ]:
parser = JsonOutputParser(pydantic_object=Book)

`JsonOutputParser` 的实例包含一个 `get_format_instructions` 方法，用于根据提供的 Pydantic 对象生成明确的 JSON 格式化指令。

In [ ]:
format_instructions = parser.get_format_instructions()

In [ ]:
print(format_instructions)

这个功能非常方便，能让解析器为我们生成详细的格式化指令。

---

## Docstrings 和 Field 描述的重要性

在上面的 `format_instructions` 中，您会注意到几个 `"description"` 字段。顶层的 `"description"` 字段是 `""关于一本书的信息""`，而 `"title"` 的 `"description"` 字段则是 `"书名"`。再看看 Pydantic 类定义...

In [ ]:
class Book(BaseModel):
    """Information about a book."""

    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    year_of_publication: str = Field(description="The year the book was published")

...您会发现这些描述是从类的文档字符串（顶层描述）以及每个字段的 `description` 值生成的。

这些文本对于向 LLM 传达我们的意图至关重要。在创建用于与 LLM 交互的 Pydantic 类时，确保为整个类提供有意义的 docstring，并为每个字段提供良好的描述。

---

## 在提示词中使用格式化指令

我们来把 `JsonOutputParser` 根据 Pydantic `Book` 类生成的格式化指令放到提示词中。顺便说一下，也可以提供一个系统消息来辅助我们。

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        "Generate JSON about the user input according to the provided format instructions.\n" +
        "Input: {input}\n" +
        "Format instructions {format_instructions}")
    )
])

接下来创建我们的链。

In [ ]:
chain = template | llm | parser # Created above with `parser = JsonOutputParser(pydantic_object=Book)`

当调用这个模板时，我们需要提供一个 `input`，这个例子中应该是一本书的标题，以及 `format_instructions`，这个已经从 `parser.format_instructions()` 中得到了。

In [ ]:
chain.invoke({
    "input": "East of Eden",
    "format_instructions": format_instructions
})

因为我们想提供不同的 `input` 值，但保持相同的 `format_instructions`，因此可以使用模板的 `.partial` 方法，将现有的 `format_instructions` 应用到提示模板中。

In [ ]:
chain = template.partial(format_instructions=format_instructions) | llm | parser # Created above with `parser = JsonOutputParser(pydantic_object=Book)`

下面用一批书试试我们的新链。

In [ ]:
book_titles = ["Dune", "Neuromancer", "Snow Crash", "The Left Hand of Darkness", "Foundation"]

In [ ]:
chain.batch(book_titles)

与之前 notebook 的输出对比（见下方），您会发现我们的结果更加一致，也更好。

```python
[{'title': 'Dune', 'author': 'Frank Herbert', 'year_of_publication': 1965},
 {'title': 'Neuromancer', 'author': 'William Gibson', 'year': 1984},
 {'title': 'Snow Crash', 'author': 'Neal Stephenson', 'yearPublished': '1992'},
 {'title': 'The Left Hand of Darkness',
  'author': None,
  'publication_year': None},
 {'title': 'Foundation', 'author': 'Isaac Asimov', 'year': '1951'}]
 ```

---

## 使用 with_structured_output

作为一种替代方案，许多 LLM 现在支持 `with_structured_output` 方法，这是一种生成结构化输出的改进方式，允许我们将以下内容...

```python
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        "Generate JSON about the user input according to the provided format instructions.\n" +
        "Input: {input}\n" +
        "Format instructions {format_instructions}")
    )
])

chain = template.partial(format_instructions=format_instructions) | llm | JsonOutputParser(pydantic_object=Book)
```

... 替换为:

```python
llm_structured = llm.with_structured_output(Book)
```

在刚才的例子中，`llm_structured` 可以像 `chain` 一样被调用、批处理或流式传输，但语法简洁得多。

截至撰写本文时（2024 年 9 月），Llama 3.1b instruct NIM 尚不支持 `with_structured_output` 方法，但我们预计它很快就会支持了。

---

## 练习：利用 Pydantic 生成结构化数据

在这个练习中，您将为以下城市生成一批结构化数据。

In [ ]:
city_names = ['Tokyo', 'Busan', 'Cairo', 'Perth']

对于这些城市，您应该创建一个包含城市信息的 JSON 对象，包括：
- 城市名称。
- 该城市所在的国家。
- 该城市是否为其所在国家的首都。
- 该城市的人口。

如果您遇到困难，可以查看下面的参考答案。

### 您的代码

### 参考答案

In [ ]:
class City(BaseModel):
    """Information about a city."""

    name: str = Field(description="The name of the city")
    country: str = Field(description="The the country the city is located in")
    capital: bool = Field(description="Is the city the capital of the country it is located in")
    population: int = Field(description="The population of the city")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI that generates JSON and only JSON according to the instructions provided to you."),
    ("human", (
        "Generate JSON about the user input according to the provided format instructions.\n" +
        "Input: {input}\n" +
        "Format instructions {format_instructions}")
    )
])

In [ ]:
parser = JsonOutputParser(pydantic_object=City)

In [ ]:
template_with_format_instructions = template.partial(format_instructions=parser.get_format_instructions())

In [ ]:
chain = template_with_format_instructions | llm | parser

In [ ]:
chain.batch(city_names)

---

## 总结

这个 notebook 中，您在生成结构化数据的能力上迈出了一大步。下一个 notebook 将进一步扩展这项技能。您不再需要提供单独的数据样本来驱动数据生成，而是将长文本输入到模型中，模型将提取并标记您指定的数据。